# Long Short-Term Memory (LSTM) for rainfall-runoff modelling

Recently, Kratzert et al. (2018a, 2018b) have shown the potential of LSTMs for rainfall-runoff modelling. Here, I'll show some example for setting up and training such a model. For single catchment calibration (as done in this notebook) no GPU is required, and everything should run fine on a standard CPU (also rather slow, compared to a GPU).

In this example, we use the NVE data set (provided by Elhadi Mohsen Hassan Abdalla) that provides us with approx. 15 years of daily meteorological forcings and discharge observations from 100 catchments across Norway. As input to our model we use daily precipitation and temperature. 

- Kratzert, F., Klotz, D., Brenner, C., Schulz, K., and Herrnegger, M.: Rainfall–runoff modelling using Long Short-Term Memory (LSTM) networks, Hydrol. Earth Syst. Sci., 22, 6005-6022, https://doi.org/10.5194/hess-22-6005-2018, 2018a. 

- Kratzert F., Klotz D., Herrnegger M., Hochreiter S.: A glimpse into the Unobserved: Runoff simulation for ungauged catchments with LSTMs, Workshop on Modeling and Decision-Making in the Spatiotemporal Domain, 32nd Conference on Neural Information Processing Systems (NeuRIPS 2018), Montréal, Canada. [https://openreview.net/forum?id=Bylhm72oKX](https://openreview.net/forum?id=Bylhm72oKX), 2018b.

- A. Newman; K. Sampson; M. P. Clark; A. Bock; R. J. Viger; D. Blodgett, 2014. A large-sample watershed-scale hydrometeorological dataset for the contiguous USA. Boulder, CO: UCAR/NCAR. https://dx.doi.org/10.5065/D6MW2F4D

This is notebook is based on: https://github.com/kratzert/pangeo_lstm_example.

Date: 18.03.2020<br/>

In [1]:
# Imports
from pathlib import Path
from typing import Tuple, List

import gcsfs
import matplotlib.pyplot as plt
from numba import njit
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import tqdm
import glob

# Globals
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # This line checks if GPU is available

file_nr = 1

## Data loading
### NVE specific data loader functions

Next define a function to load the meteorological forcings and the discharge for any specific catchment from the NVE data set.

In [2]:
def load_forcing_and_discharge(catchment: int) -> Tuple[pd.DataFrame, int]:
    """Load the meteorological forcing data of a specific catchment.

    :param catchment: number (id)
    
    :return: pd.DataFrame containing the meteorological forcing data.
    """
    path = '../Input files (.txt)'
    all_files = glob.glob(path + "/*.txt")

    file_exist = False
    
    # Loop through files and find correct catchment
    for file_path in all_files:
        # Name is formatted `./Input files (.txt)/nve_inp_XX.txt`
        number = int(file_path.split('_')[-1].split('.')[0])
        
        if number == catchment:
            file_exist = True
            df = pd.read_csv(file_path, encoding='cp1252', skiprows=[0], delimiter=r"\s+", parse_dates=[['dd.mm.yyyy',  'hh:mm:ss']])
            df = df.rename(columns={"dd.mm.yyyy_hh:mm:ss": "timestamp"})
    
    # Return None if catchment does not exist
    if file_exist == False:
        print("Catchment does not exist")
        return None
    else:
        return df

In [3]:
# Display Data frame for one catchment
display(load_forcing_and_discharge(file_nr))

,timestamp,mm,grC,grC.1,m3/s
0,2000-01-01,3.30,-3.75,-3.75,0.27
1,2000-02-01,3.79,-2.02,-2.02,0.27
2,2000-03-01,6.98,-3.84,-3.84,0.27
3,2000-04-01,16.53,-3.80,-3.80,0.29
4,2000-05-01,4.44,-4.78,-4.78,0.31
...,...,...,...,...,...
5839,2015-12-27,2.27,-13.05,-13.05,1.79
5840,2015-12-28,1.59,-11.45,-11.45,2.57
5841,2015-12-29,0.38,-9.41,-9.41,2.62
5842,2015-12-30,1.86,-5.59,-5.59,2.16


## Import training data

In [4]:
def load_training_data(catchment: int) -> Tuple[pd.DataFrame, int]:
    """Load the meteorological forcing data and other characteristics of a specific catchment.

    :param catchment: number (id)
    
    :return: pd.DataFrame containing the meteorological forcing and other characteristics data.
    """
    path = '../Training_data'
    all_files = glob.glob(path + "/*.csv")

    file_exist = False
    
    # Loop through files and find correct catchment
    for file_path in all_files:
        # Name is formatted `./Input files (.txt)/nve_inp_XX.txt`
        number = int(file_path.split('_')[-1].split('.')[0])
        
        if number == catchment:
            file_exist = True
            df = pd.read_csv(file_path)
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            # Delay by one day
            df["Residual_in"] = np.roll(df["Residual"], 1)
            
    # Return None if catchment does not exist
    if file_exist == False:
        print("Catchment does not exist")
        return None
    else:
        return df

In [5]:
# Display training data frame for one catchment
display(load_training_data(file_nr))

,Unnamed: 0,timestamp,mm,grC,grC.1,m3/s,evapo_transp,grC1,grC2,grC3,...,mm6,mm7,mm8,mm9,mm10,EPOT,Q_obs,Q_sim,Residual,Residual_in
0,0,2000-01-01,3.30,-3.75,-3.75,0.27,0.1,-5.784545,-4.902045,-4.364545,...,9.735233,9.907284,10.039809,10.170009,10.572236,0.1,0.27,0.006,-0.264,0.032
1,1,2000-01-02,3.79,-2.02,-2.02,0.27,0.1,-4.054545,-3.172045,-2.634545,...,11.180767,11.378365,11.530569,11.680102,12.142053,0.1,0.27,0.025,-0.245,-0.264
2,2,2000-01-03,6.98,-3.84,-3.84,0.27,0.1,-5.874545,-4.992045,-4.454545,...,20.591493,20.955406,21.235717,21.511111,22.361881,0.1,0.27,0.047,-0.223,-0.245
3,3,2000-01-04,16.53,-3.80,-3.80,0.29,0.1,-5.834545,-4.952045,-4.414545,...,48.764667,49.626484,50.290316,50.942502,52.957291,0.1,0.29,0.080,-0.210,-0.223
4,4,2000-01-05,4.44,-4.78,-4.78,0.31,0.1,-6.814545,-5.932045,-5.394545,...,13.098313,13.329800,13.508107,13.683285,14.224463,0.1,0.31,0.100,-0.210,-0.210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5839,5839,2015-12-27,2.27,-13.05,-13.05,1.79,0.1,-15.084545,-14.202045,-13.664545,...,6.696660,6.815010,6.906172,6.995734,7.272417,0.1,1.79,1.346,-0.444,-0.111
5840,5840,2015-12-28,1.59,-11.45,-11.45,2.57,0.1,-13.484545,-12.602045,-12.064545,...,4.690612,4.773509,4.837363,4.900095,5.093895,0.1,2.57,1.289,-1.281,-0.444
5841,5841,2015-12-29,0.38,-9.41,-9.41,2.62,0.1,-11.444545,-10.562045,-10.024545,...,1.121027,1.140839,1.156099,1.171092,1.217409,0.1,2.62,1.232,-1.388,-1.281
5842,5842,2015-12-30,1.86,-5.59,-5.59,2.16,0.1,-7.624545,-6.742045,-6.204545,...,5.487131,5.584105,5.658801,5.732187,5.958897,0.1,2.16,1.179,-0.981,-1.388


### Data reshaping for LSTM training
Next we need another utility function to reshape the data into an appropriate format for training LSTMs. These recurrent neural networks expect sequential input of the shape `(sequence length, number of features)`. We train our network to predict a single day of discharge from *n* days of precendent meteorological observations. For example, lets assume that _n_ = 365, then a single training sample should be of shape `(365, number of features)`, and since we use 2 input features the shape is `(365, 2)`. Here the features are temperature and rain fall.

However, when loaded from the files the entire data is stored in a matrix, where the number of rows correspond to the total number of days in the training set and the number of columns to the features. Thus we need to slide over this matrix and cut out small samples appropriate to our LSTM setting. To speed things up, we make use of the awesome Numba library here (the little @njit decorator JIT-compiles this function and dramatically increases the speed).

In [6]:
@njit
def reshape_data(x: np.ndarray, y: np.ndarray, seq_length: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Reshape matrix data into sample shape for LSTM training.

    :param x: Matrix containing input features column wise and time steps row wise
    :param y: Matrix containing the output feature.
    :param seq_length: Length of look back days for one day of prediction
    
    :return: Two np.ndarrays, the first of shape (samples, length of sequence,
        number of features), containing the input data for the LSTM. The second
        of shape (samples, 1) containing the expected output for each input
        sample.
    """
    num_samples, num_features = x.shape

    x_new = np.zeros((num_samples - seq_length + 1, seq_length, num_features))
    y_new = np.zeros((num_samples - seq_length + 1, 1))

    for i in range(0, x_new.shape[0]):
        x_new[i, :, :num_features] = x[i:i + seq_length, :]
        y_new[i, :] = y[i + seq_length - 1, 0]

    return x_new, y_new

### PyTorch data set
Now we wrap everything into a PyTorch Dataset. These are specific classes that can be used by PyTorchs DataLoader class for generating mini-batches (and do this in parallel in multiple threads). Such a data set class has to inherit from the PyTorch Dataset class and three functions have to be implemented

1. __init__(): The object initializing function.
2. __len__(): This function has to return the number of samples in the data set.
3. __getitem__(i): A function that returns sample `i` of the data set (sample + target value).

In [7]:
class NVE_TXT(Dataset):
    """Torch Dataset for basic use of data from the NVE data set.

    This data set provides meteorological observations and discharge of a given
    catchment from the NVE data set.
    """

    def __init__(self, catchment: int, seq_length: int=365,period: str=None,
                 dates: List=None, means: pd.Series=None, stds: pd.Series=None):
        """Initialize Dataset containing the data of a single catchment.

        :param catchment: number (id) indicating a given NVE catchment 
        :param seq_length: (optional) Length of the time window of
            meteorological input provided for one time step of prediction.
        :param period: (optional) One of ['train', 'eval']. None loads the 
            entire time series.
        :param dates: (optional) List of pd.DateTimes of the start and end date 
            of the discharge period that is used.
        :param means: (optional) Means of input and output features derived from
            the training period. Has to be provided for 'eval' period. Can be
            retrieved if calling .get_means() on the data set.
        :param stds: (optional) Stds of input and output features derived from
            the training period. Has to be provided for 'eval' period. Can be
            retrieved if calling .get_stds() on the data set.
        """
        self.catchment = catchment
        self.seq_length = seq_length
        self.period = period
        self.dates = dates
        self.means = means
        self.stds = stds
            
        # load data into memory
        self.x, self.y = self._load_data()

        # store number of samples as class attribute
        self.num_samples = self.x.shape[0]

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx: int):
        return self.x[idx], self.y[idx]

    def _load_data(self):
        """Load input and output data from text files."""
        #df = load_forcing_and_discharge(self.catchment)
        df = load_training_data(self.catchment)
        if self.dates is not None:
            # Truncate data after start and end date
            df = df[df['timestamp'] >= self.dates[0]]
            df = df[df['timestamp'] < self.dates[1]]
        
        # if training period store means and stds
        if self.period == 'train':
            self.means = df.mean()
            self.stds = df.std()         
        
        # extract input and output features from DataFrame
        x = np.array([df['mm1'].values,
                      df['mm2'].values,
                      df['mm3'].values,
                      df['mm4'].values,
                      df['mm5'].values,
                      df['mm6'].values,
                      df['mm7'].values,
                      df['mm8'].values,
                      df['mm9'].values,
                      df['mm10'].values,
                      df['grC1'].values,
                      df['grC2'].values,
                      df['grC3'].values,
                      df['grC4'].values,
                      df['grC5'].values,
                      df['grC6'].values,
                      df['grC7'].values,
                      df['grC8'].values,
                      df['grC9'].values,
                      df['grC10'].values,
                      df['Residual_in'].values,
                      df['EPOT'].values]).T
        y = np.array([df['Residual'].values]).T
        
        # normalize data, reshape for LSTM training and remove invalid samples
        x = self._local_normalization(x, variable='inputs')
        x, y = reshape_data(x, y, self.seq_length)                      

        if self.period == "train":            
            # normalize discharge
            y = self._local_normalization(y, variable='output')

        # convert arrays to torch tensors
        x = torch.from_numpy(x.astype(np.float32))
        y = torch.from_numpy(y.astype(np.float32))

        return x, y

    def _local_normalization(self, feature: np.ndarray, variable: str) -> \
            np.ndarray:
        """Normalize input/output features with local mean/std.

        :param feature: Numpy array containing the feature(s) as matrix.
        :param variable: Either 'inputs' or 'output' showing which feature will
            be normalized
        :return: array containing the normalized feature
        """
        if variable == 'inputs':
            means = np.array([self.means['mm1'],
                              self.means['mm2'],
                              self.means['mm3'],
                              self.means['mm4'],
                              self.means['mm5'],
                              self.means['mm6'],
                              self.means['mm7'],
                              self.means['mm8'],
                              self.means['mm9'],
                              self.means['mm10'],
                              self.means['grC1'],
                              self.means['grC2'],
                              self.means['grC3'],
                              self.means['grC4'],
                              self.means['grC5'],
                              self.means['grC6'],
                              self.means['grC7'],
                              self.means['grC8'],
                              self.means['grC9'],
                              self.means['grC10'],
                              self.means['Residual_in'],
                              self.means['EPOT']])
            stds = np.array([self.stds['mm1'],
                             self.stds['mm2'],
                             self.stds['mm3'],
                             self.stds['mm4'],
                             self.stds['mm5'],
                             self.stds['mm6'],
                             self.stds['mm7'],
                             self.stds['mm8'],
                             self.stds['mm9'],
                             self.stds['mm10'],
                             self.stds['grC1'],
                             self.stds['grC2'],
                             self.stds['grC3'],
                             self.stds['grC4'],
                             self.stds['grC5'],
                             self.stds['grC6'],
                             self.stds['grC7'],
                             self.stds['grC8'],
                             self.stds['grC9'],
                             self.stds['grC10'],
                             self.stds['Residual_in'],
                             self.stds['EPOT']])
            feature = (feature - means) / stds
        elif variable == 'output':
            feature = ((feature - self.means["Residual"]) /
                       self.stds["Residual"])
        else:
            raise RuntimeError(f"Unknown variable type {variable}")

        return feature
    
    def local_rescale(self, feature: np.ndarray, variable: str) -> \
            np.ndarray:
        """Rescale input/output features with local mean/std.

        :param feature: Numpy array containing the feature(s) as matrix.
        :param variable: Either 'inputs' or 'output' showing which feature will
            be normalized
        :return: array containing the normalized feature
        """
        if variable == 'inputs':
            means = np.array([self.means['mm1'],
                              self.means['mm2'],
                              self.means['mm3'],
                              self.means['mm4'],
                              self.means['mm5'],
                              self.means['mm6'],
                              self.means['mm7'],
                              self.means['mm8'],
                              self.means['mm9'],
                              self.means['mm10'],
                              self.means['grC1'],
                              self.means['grC2'],
                              self.means['grC3'],
                              self.means['grC4'],
                              self.means['grC5'],
                              self.means['grC6'],
                              self.means['grC7'],
                              self.means['grC8'],
                              self.means['grC9'],
                              self.means['grC10'],
                              self.means['Residual_in'],
                              self.means['EPOT']])
            stds = np.array([self.stds['mm1'],
                             self.stds['mm2'],
                             self.stds['mm3'],
                             self.stds['mm4'],
                             self.stds['mm5'],
                             self.stds['mm6'],
                             self.stds['mm7'],
                             self.stds['mm8'],
                             self.stds['mm9'],
                             self.stds['mm10'],
                             self.stds['grC1'],
                             self.stds['grC2'],
                             self.stds['grC3'],
                             self.stds['grC4'],
                             self.stds['grC5'],
                             self.stds['grC6'],
                             self.stds['grC7'],
                             self.stds['grC8'],
                             self.stds['grC9'],
                             self.stds['grC10'],
                             self.stds['Residual_in'],
                             self.stds['EPOT']])
            feature = feature * stds + means
        elif variable == 'output':
            feature = (feature * self.stds["Residual"] +
                       self.means["Residual"])
        else:
            raise RuntimeError(f"Unknown variable type {variable}")

        return feature

    def get_means(self):
        return self.means

    def get_stds(self):
        return self.stds


## Build LSTM model

Here we implement a single layer LSTM with optional dropout in the final fully connected layer. Using PyTorch, we need to inherit from the `nn.Module` class and implement the `__init__()`, as well as a `forward()` function. To make things easy, we make use of the standard LSTM layer included in the PyTorch library.

The forward function implements the entire forward pass through the network, while the backward pass, used for updating the weights during training, is automatically derived from PyTorch autograd functionality.

In [8]:
class Model(nn.Module):
    """Implementation of a single layer LSTM network"""
    
    def __init__(self, hidden_size: int, dropout_rate: float=0.0):
        """Initialize model
        
        :param hidden_size: Number of hidden units/LSTM cells
        :param dropout_rate: Dropout rate of the last fully connected
            layer. Default 0.0
        """
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        # create required layer
        self.lstm = nn.LSTM(input_size=22, hidden_size=self.hidden_size, 
                            num_layers=1, bias=True, batch_first=True)
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.fc = nn.Linear(in_features=self.hidden_size, out_features=1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass through the Network.
        
        :param x: Tensor of shape [batch size, seq length, num features]
            containing the input data for the LSTM network.
        
        :return: Tensor containing the network predictions
        """
        output, (h_n, c_n) = self.lstm(x)
        
        # perform prediction only at the end of the input sequence
        pred = self.fc(self.dropout(h_n[-1,:,:]))
        return pred

## Train utilities

Next we are implementing two functions for training and evaluating the model.

- `train_epoch()`: This function iterates one time over the entire training data set (called one epoch) and updates the weights of the network to minimize the loss function (we use the mean squared error here).
- `eval_model()`: To this function evaluates a data set and returns the predictions, as well as observations.

Furthermore we implement a function `calc_nse()` to calculate the Nash-Sutcliffe-Efficiency for our model

In [9]:
def train_epoch(model, optimizer, loader, loss_func, epoch):
    """Train model for a single epoch.

    :param model: A torch.nn.Module implementing the LSTM model
    :param optimizer: One of PyTorchs optimizer classes.
    :param loader: A PyTorch DataLoader, providing the trainings
        data in mini batches.
    :param loss_func: The loss function to minimize.
    :param epoch: The current epoch (int) used for the progress bar
    """
    # set model to train mode (important for dropout)
    model.train()
    pbar = tqdm.tqdm_notebook(loader)
    pbar.set_description(f"Epoch {epoch}")
    # request mini-batch of data from the loader
    for xs, ys in pbar:
        # delete previously stored gradients from the model
        optimizer.zero_grad()
        # push data to GPU (if available)
        xs, ys = xs.to(DEVICE), ys.to(DEVICE)
        # get model predictions
        y_hat = model(xs)
        # calculate loss
        loss = loss_func(y_hat, ys)
        # calculate gradients
        loss.backward()
        # update the weights
        optimizer.step()
        # write current loss in the progress bar
        pbar.set_postfix_str(f"Loss: {loss.item():.4f}")

        
def eval_model(model, loader) -> Tuple[torch.Tensor, torch.Tensor]:
    """Evaluate the model.

    :param model: A torch.nn.Module implementing the LSTM model
    :param loader: A PyTorch DataLoader, providing the data.
    
    :return: Two torch Tensors, containing the observations and 
        model predictions
    """
    # set model to eval mode (important for dropout)
    model.eval()
    obs = []
    preds = []
    # in inference mode, we don't need to store intermediate steps for
    # backprob
    with torch.no_grad():
        # request mini-batch of data from the loader
        for xs, ys in loader:
            # push data to GPU (if available)
            xs = xs.to(DEVICE)
            # get model predictions
            y_hat = model(xs)
            obs.append(ys)
            preds.append(y_hat)
            
    return torch.cat(obs), torch.cat(preds)
        
def calc_nse(obs: np.array, sim: np.array) -> float:
    """Calculate Nash-Sutcliff-Efficiency.

    :param obs: Array containing the observations
    :param sim: Array containing the simulations
    :return: NSE value.
    """
    # only consider time steps, where observations are available
    sim = np.delete(sim, np.argwhere(obs < 0), axis=0)
    obs = np.delete(obs, np.argwhere(obs < 0), axis=0)

    # check for NaNs in observations
    sim = np.delete(sim, np.argwhere(np.isnan(obs)), axis=0)
    obs = np.delete(obs, np.argwhere(np.isnan(obs)), axis=0)

    denominator = np.sum((obs - np.mean(obs)) ** 2)
    numerator = np.sum((sim - obs) ** 2)
    nse_val = 1 - numerator / denominator

    return nse_val

## Prepare everything for training

Now that we have everything needed, we have to do the following steps:

- Specify a training, as well independent validation and test periods. For each of the three create a PyTorch data set and with these data sets PyTorch DataLoader. These DataLoader put single samples together to mini-batches that we use to train the network.
- Initialize the model
- Create a optimizer
- Define the loss function

In [10]:
catchment = file_nr # can be changed to any catchment id contained in the NVE data set
hidden_size = 10 # Number of LSTM cells
dropout_rate = 0.0 # Dropout rate of the final fully connected Layer [0.0, 1.0]
learning_rate = 10*1e-3 # Learning rate used to update the weights
sequence_length = 365 # Length of the meteorological record provided to the network

##############
# Data set up#
##############

# Training data
start_date = pd.to_datetime("2000-01-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2007-08-31", format="%Y-%m-%d")
ds_train = NVE_TXT(catchment, seq_length=sequence_length, period="train", dates=[start_date, end_date])
tr_loader = DataLoader(ds_train, batch_size=256, shuffle=True)
print(f"Training data length: {len(ds_train)}")

# Validation data. We use the feature means/stds of the training period for normalization
means = ds_train.get_means()
stds = ds_train.get_stds()
start_date = pd.to_datetime("2008-01-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2009-08-31", format="%Y-%m-%d")
ds_val = NVE_TXT(catchment, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds)
val_loader = DataLoader(ds_val, batch_size=2048, shuffle=False)
print(f"Validation data length: {len(ds_val)}")

# Test data. We use the feature means/stds of the training period for normalization
start_date = pd.to_datetime("2010-01-01", format="%Y-%m-%d")
end_date = pd.to_datetime("2015-08-31", format="%Y-%m-%d")
ds_test = NVE_TXT(catchment, seq_length=sequence_length, period="eval", dates=[start_date, end_date],
                     means=means, stds=stds)
test_loader = DataLoader(ds_test, batch_size=2048, shuffle=False)
print(f"Test data length: {len(ds_test)}")

#########################
# Model, Optimizer, Loss#
#########################

# Here we create our model, feel free 
model = Model(hidden_size=hidden_size, dropout_rate=dropout_rate).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.MSELoss()

Training data length: 2435
Validation data length: 244
Test data length: 1704


## Train the model
Now we gonna train the model for some number of epochs. After each epoch we evaluate the model on the validation period and print out the NSE.

In [11]:
n_epochs = 10 # Number of training epochs

for i in range(n_epochs):
    train_epoch(model, optimizer, tr_loader, loss_func, i+1)
    obs, preds = eval_model(model, val_loader)
    preds = ds_val.local_rescale(preds.numpy(), variable='output')
    nse = calc_nse(obs.numpy(), preds)
    tqdm.tqdm.write(f"Validation NSE: {nse:.3f}")

<ipython-input-9-ed41dce149ba>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm.tqdm_notebook(loader)



Validation NSE: -0.157



Validation NSE: 0.040



Validation NSE: 0.255



Validation NSE: 0.355



Validation NSE: 0.331



Validation NSE: 0.380



Validation NSE: 0.249



Validation NSE: 0.344



Validation NSE: 0.275



Validation NSE: 0.314


## Evaluate independent test set
Finally, we can can evaluate our model on the unseen test data, calculate the NSE and plot some observations vs predictions

<Figure size 1440x1152 with 0 Axes>

In [12]:
# Evaluate on test set
obs, preds = eval_model(model, test_loader)
preds = ds_val.local_rescale(preds.numpy(), variable='output')
obs = obs.numpy()
nse = calc_nse(obs, preds)

# Plot results
fig, ax = plt.subplots(figsize=(12, 4))

#start_date = ds_test.dates[0]
#end_date = ds_test.dates[1] + pd.DateOffset(days=1)
#date_range = pd.date_range(start_date, end_date)

plt.figure(figsize=(20, 16))
ax.plot(obs, label="observation")
ax.plot(preds, label="prediction")
ax.legend()

ax.set_title(f"Catchment {catchment} - Test set NSE: {nse:.3f}")
ax.xaxis.set_tick_params(rotation=90)
ax.set_xlabel("Date")
_ = ax.set_ylabel("Discharge (m3/s)")